In [4]:
import numpy as np
import pandas as pd

import os
import glob
import librosa
import tqdm
import pickle

DATA_PATH = './data'
join_path = lambda path: os.path.join(DATA_PATH, path)

In [5]:
sampling_rate = 44100 # от оргов контеста
duration = 2 # сколько секунд от записи хотим взять
n_samples = sampling_rate * duration

def get_raw(filepaths, verbose=False):
    data = []
    for fp in tqdm.tqdm(filepaths, disable=not verbose):
        y, _ = librosa.load(fp, sr=sampling_rate)
        y = np.concatenate([[0.] * (n_samples - len(y)), y[:n_samples]])
        data.append(y.astype(np.float32))
    data = np.array(data)
    return data

In [7]:
train_filepaths = glob.glob(join_path('split/train/*'))
val_filepaths = glob.glob(join_path('split/val/*'))

train_data = get_raw(train_filepaths, verbose=True)
with open(join_path('train_raw.pkl'), 'wb') as f:
    pickle.dump(train_data, f, pickle.HIGHEST_PROTOCOL)

val_data = get_raw(val_filepaths, verbose=True)
with open(join_path('val_raw.pkl'), 'wb') as f:
    pickle.dump(val_data, f, pickle.HIGHEST_PROTOCOL)

100%|███████████████████████████████████████| 506/506 [00:02<00:00, 171.80it/s]


In [8]:
def get_targets(filepaths):
    
    targets = np.array(
        train.loc[[get_filename(fp) for fp in filepaths], 'labels'] \
            .str \
            .split(',') \
            .apply(
                lambda x: [int(label in x) for label in cats]
            ) \
            .tolist()
    )
    
    return targets

get_filename = lambda x: x[-12:]

train = pd.read_csv(join_path('train_curated.csv')).set_index('fname')

cats = list(set(sum(list(train['labels'].str.split(',').values), [])))

train_targets = get_targets(train_filepaths)
val_targets = get_targets(val_filepaths)

np.save(join_path('train_targets.npy'), train_targets)
np.save(join_path('val_targets.npy'), val_targets)